# Project: Data Wrangling and Analysis of WeRateDogs Twitter Archive.

## Table of Contents
* [Data Gathering](#data-gathering)
* [Data Assessing](#assessing-data)
* [Data Cleaning](#cleaning-data)
* [Data Visualizing](#analyzing-and-visualizing-data)

In [1]:
# importing packages for this project
import pandas as pd #for data wrangling
import numpy as np # for mathematical computing
import requests # for downloading files programmatically
import os # for accessing downloaded files
import tweepy # to query twitter API
import json # to write a json data off the querried data#
import time # time module allows to work with time#
import matplotlib.pyplot as plt #for visualization
%matplotlib inline #for visualization
import seaborn as sns #for visualization
import sklearn
import datetime
import io

UsageError: unrecognized arguments: #for visualization


## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
# reading the downloaded file to pandas 
twitter_archive = pd.read_csv('../data/raw_data/twitter-archive-enhanced.csv')

In [ ]:
# view read file
twitter_archive

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
# creating a request for image_predictions file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

# accessing the content of downloaded file and writing to a file
with open(os.path.join('../data/raw_data/image-predictions.tsv'), mode = 'wb') as file:
    file.write(response.content)

# reading image predictions file to pandas
image_predictions = pd.read_csv('../data/raw_data/image-predictions.tsv', sep='\t')

In [ ]:
# view image_predictions file
image_predictions

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
from tweepy import OAuthHandler
from timeit import default_timer as timer

In [ ]:
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)

In [ ]:
# to extract the 'id', 'retweet_count', 'favorite_count', 'followers_count',
# 'friends_count', 'listed_count' from 'tweet_json'
# and later convert to a DataFrame

#create and empty list to house the extracted data
df_list =[]

# open .txt file for reading.
with open ('../data/tweet_json.txt', 'r') as jsonfile:
    for line in jsonfile.readlines():
         # read json string into a dictionary
        tweet_line = json.loads(line)
        # getting the required parameters
        tweet_ID = tweet_line['id']
        retweet_count = tweet_line['retweet_count']
        friends_count = tweet_line['user']['friends_count']
        fav_count = tweet_line['favorite_count']
        followers_count = tweet_line['user']['followers_count']
        listed_count = tweet_line['user']['listed_count']
        
        
        # Append to list of dictionaries
        df_list.append({'id': tweet_ID,
                       'retweet_count': retweet_count,
                       'friends_count': friends_count,
                       'favorite_count': fav_count,
                       'followers_count': followers_count,
                       'listed_count': listed_count})
        
# creating a dataframe off the dictionaries
tweet_json = pd.DataFrame(df_list, columns=['id', 'retweet_count',
                                            'friends_count',
                                            'favorite_count', 'followers_count', 
                                            'listed_count'])    

In [ ]:
df_list

In [ ]:
tweet_json

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [ ]:
# for easy assess, rename the dataframes
df1 = twitter_archive
df2 = image_predictions
df3 = tweet_json

### Visual Assessment:
Here, a directed visual assessment of the dataframes will be carried out, aiming to explain the columns and check for anomalous data.

In [ ]:
df1.head(3)

**`df1`** columns: 

1. **`tweet_id`**: this's the unique tweet identifier
2. **`in_reply_to_status_id`**: contains the integer representation of the original tweet's ID, if the `tweet_id` is a reply.
3. **`in_reply_to_user_id`**: if `tweet_id` is a reply, this contains the integer representation of the original Tweet's author ID.
4. **`timestamp`**: contains time when the tweet was created.
5. **`source`**: contains a display of the devices through which the tweet was created.
6. **`text`**: the text element of the tweet
7. **`retweeted_status_id`**: contains interger representation of the original `tweet_id` if `tweet_id` is a retweet.
8. **`retweeted_status_user_id`**: if `tweet_id` is a retweet, this displays the integer representaion of the original Tweet's author ID.
9. **`retweeted_status_timestamp`**: time of retweet.
10. **`expanded_urls`**: tweet's URL.
11. **`rating_numerator`**: conatains the numerator of the rating of a dog. ratings are almost always greater than 10.
12. **`name`**: name of the dog.

In [ ]:
df2.head(3)

**`df2`** columns:

1. **`tweet_id`**: the unique identifier for each tweet
2. **`jpg_url`**: dog's image URL
3. **`img_num`**: the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).
4. **`p1`**: algorithm's #1 prediction for the image in the tweet
5. **`p1_conf`**: how confident the algorithm is in its #1 prediction.
6. **`p1_dog`**: whether or not the #1 prediction is a breed of dog
7. **`p2`**: algorithm's #2 prediction for the image in the tweet
8. **`p2_conf`**: how confident the algorithm is in its #2 prediction.
9. **`p2_dog`**: whether or not the #2 prediction is a breed of dog.
10. **`p3`**: algorithm's #3 prediction for the image in the tweet.
11. **`p3_conf`**: how confident the algorithm is in its #3 prediction.
12. **`p3_dog`**: whether or not the #3 prediction is a breed of dog

In [ ]:
df3.head(4)

**`df3`** columns:

1. **`id`**: the unique identifier for each tweet.
2. **`retweet_count`**: the number of times the original tweet was retweeted.
3. **`favorite_count`**: the number of times the the original tweet was loved or liked.
4. **`followers_count`**: the number of followers of WeRataeDogs account as at the time of the each tweet.
5. **`friends_count`**: the number of profiles WeRateDogs account was following at the time of each tweet.
6. **`listed_count`**: The number of public lists that this user is a member of.

### Programmatic Assessment
Here, directed assessment using different pandas function will be used to assess the three(3) dataframe,

First, we use the `.info()` function to get a summary of the dataframe

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df3.info()

**Looking at the statistical description of the DataFrames:**

In [ ]:
df1.describe()

In [ ]:
df2.describe()

In [ ]:
df3.describe()

**Checking for `null` values in the DataFrame**

In [ ]:
df1.isna().sum()

In [ ]:
df2.isna().sum()

In [ ]:
 df3.isna().sum()

**Checking for duplicated values in the DataFrames**

In [ ]:
df1.duplicated().sum()

In [ ]:
df2.duplicated().sum()

In [ ]:
df3.duplicated().sum()

In [ ]:
df1.name.value_counts().head(50)

### Quality issues
#### Twitter_archive (df1)
1. **`Tweet_id`** should be a string not integer
2. **`Timestamp`** is Datetime not object
3. **`Tweet_id`** with **`Retweeted_status_id`**, **`Retweeted_status_user_id`**, **`Retweeted_status_timestamp`** should be dropped.
4. **`Retweeted_status_timestamp`** is datetime not object.
5. Missing values in **`Retweeted_status_id`**
6. Missing values in **`Retweeted_status_user_id`**
7. Missing values in **`Retweeted_status_timestamp`**
8. Missing values in **`Expanded_urls`**
9. Missing values in **`in_Reply_To_Status_Id`**
10. Missing values in **`In_Reply_To_User_id`**
11. **`name`** value contains invalid names in lower case.
12. Take out url at the end of some strings in `text column`

#### Image_Predictions (df2)
13. **`Tweet_id`** has wrong datatype.

#### Tweet_json
14. **`id`** has wrong datatype.
15. wrong column name **`id`** 

### Tidiness issues
#### Twitter_archive (df1)
15. `friends_count` column has only one value (104).
16. Rating should be in a single column
17. **`doggo`**, **`floofer`**, **`pupper`** and **`puppo`** columns need to be one column **`dog_stage`** with 4 categories.

### General Issue:

* Merging tables

<a id='Data Cleaning'></a>
## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data
df1_clean = df1.copy()
df2_clean = df2.copy()
df3_clean = df3.copy()

### Issue #1: datatype issues

#### Define:
* In df1 change **timestamp** datatype from object to datetime.
* In df1 change **tweet_id** datatype  from int to string.
* In df1 change **Retweeted_status_timestamp** datatype from object to datetime.
* In df2 change **tweet_id** from int to string.
* In df3 change **id** datatype from int to string.

#### Code

In [ ]:
# use the datetime class of the datetime module for this conversion
df1_clean['timestamp'] = pd.to_datetime(df1_clean['timestamp'])

# change tweet_id column from int to string
df1_clean['tweet_id'] = df1_clean['tweet_id'].astype(str)

# change Retweeted_status_timestamp datatype from object to datetime.
df1_clean['retweeted_status_timestamp'] = pd.to_datetime(df1_clean['retweeted_status_timestamp'])

# In df2 change **tweet_id** from int to string.
df2_clean['tweet_id'] = df2_clean['tweet_id'].astype(str)

# In df3 change **id** datatype from int to string.
df3_clean['id'] = df3_clean['id'].astype(str)

#### Test

In [ ]:
# use dtype method to check df1_clean
df1_clean.dtypes

In [ ]:
# use dtype method to check df2_clean
df2_clean.dtypes

In [ ]:
# use dtype method to check df3
df3_clean.dtypes

### Issue #2: Erroneous IDs in df1 tweet_id

#### Define
remove rows with values for 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' in Df1. These IDs are that of retweets and won't be used for our analysis.

#### Code

In [ ]:
# Drop retweeted rows
df1_clean = df1_clean[df1_clean.retweeted_status_id.isnull()]
df1_clean = df1_clean[df1_clean.retweeted_status_user_id.isnull()]
df1_clean = df1_clean[df1_clean.retweeted_status_timestamp.isnull()]

#### Test

In [ ]:
df1_clean.info()

### Issue 3: Missing values

* Missing values in Retweeted_status_id
* Missing values in Retweeted_status_user_id
* Missing values in Retweeted_status_timestamp
* Missing values in Expanded_urls
* Missing values in in_Reply_To_Status_Id
* Missing values in In_Reply_To_User_id

#### Define:
* drop the above columns from df1_clean as they aren't needed in EDA

#### Code:

In [ ]:
# Drop the columns
df1_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id',
                        'retweeted_status_id', 'retweeted_status_user_id',
                        'retweeted_status_timestamp', 'expanded_urls'], inplace=True)
                        

#### Test

In [ ]:
# check df1_clean fro cleaning task
df1_clean.info()

In [ ]:
# confirm that there's no missing value
df1_clean.isnull().sum()

### Issue 4

* `name` value contains invalid names in lower case

#### Define:

* Access and drop all values in the name column that started with small letters.

#### Code: 

In [ ]:
# first, create a list that contains all lowercase names
lowercase_names = []

# create a for loop to gather all lowercase names.
for name in df1_clean.name:
    if name.islower() and name not in lowercase_names:
        lowercase_names.append(name)
        
print(lowercase_names)

In [ ]:
# Replace 'lowercase_names' with 'none' in df1_clean
df1_clean.name.replace(lowercase_names, 'None', inplace=True)

#### Test

In [ ]:
# to test if lowercase names still exists in df1_clean.name
# using a for loop
for names in df1_clean.name:
    if names.islower():
        print(names)
        
# print should return nothing.

### Issue 5

* wrong column name `id` in df3

#### Define:
* change `id` in df3 to `tweet_id` like other dataframes

#### Code

In [ ]:
# use the rename method to change column head.
df3_clean.rename(columns={'id': 'tweet_id'}, inplace=True)

#### Test

In [ ]:
df3_clean.head(3)

### Issues 6

* `friends_count` has single value

#### Define
* `friends_count`in df3 is a constant and wont be needed in this analysis, so will be dropped

#### Code

In [ ]:
# using the drop() function to drop the column
df3_clean.drop('friends_count', axis=1, inplace=True)

#### Test

In [ ]:
df3_clean.info()

### Issue 7: 

* Take out url at the end of some strings in text column

#### Define:

* use the replace function and strip function on the `text` column to remove the url and whitespace.

#### Code

In [ ]:
df1_clean['text'] = df1_clean.text.str.replace(r"https\S+", "")
df1_clean['text'] = df1_clean.text.str.strip()

#### Test

In [ ]:
# querrying for http to check effect of last line.
df1_clean.query("text == 'http'")
# this should return and empty column

### Issues 8

* Rating should be in a single column

#### Define

*Create a new column called `rating` that will contain the expression: `rating_numerator/rating_denominator` on the df1_clean 
* drop `rating_numerator and denominator` column

#### Code

In [ ]:
# creating rating column with its content
df1_clean["rating"] = df1_clean["rating_numerator"]/df1_clean["rating_denominator"]

# dropping rating denominator and rating numerator columns
df1_clean = df1_clean.drop(["rating_numerator", "rating_denominator"], axis=1)

#### Test

In [ ]:
# check for new rating column
print(df1_clean.rating.sample(20))

# check for drooped columns
print(df1_clean.info())

### Issue 9: Tidiness Issues

* doggo, floofer, pupper and puppo columns need to be one column dog_stage with 4 categories.

#### Define

* replace empty string rows in `doggo`, `floofer`, `pupper` and `puppo` with None.
* combine the four columns to form a `dog_stage column`
* use querries to replace mixed dog stage
* use the drop() function to drop the 4 columns

#### Code

In [ ]:
# First replace replace empty string rows in 
# `doggo`, `floofer`, `pupper` and `puppo` with None.
#using the replace() function
df1_clean.doggo.replace('None', '', inplace=True)
df1_clean.floofer.replace('None', '', inplace=True)
df1_clean.pupper.replace('None', '', inplace=True)
df1_clean.puppo.replace('None', '', inplace=True)

# combine the four columns to form a `dog_stage column`
df1_clean['dog_stage'] = df1_clean.doggo + df1_clean.floofer + df1_clean.pupper + df1_clean.puppo


# use querries to format mixed dog stage 
df1_clean.loc[df1_clean.dog_stage == "'doggopupper', 'dog_stage'"] = 'doggo,pupper'
df1_clean.loc[df1_clean.dog_stage == "'doggopuppo', 'dog_stage'"] = 'doggo,puppo'
df1_clean.loc[df1_clean.dog_stage == "'doggofloofer', 'dog_stage'"] = 'doggo,floofer'

# use the drop() function to drop the 4 columns
df_drop = ['doggo', 'floofer', 'pupper', 'puppo']
df1_clean.drop(df_drop, axis=1, inplace=True)

In [ ]:
(df1_clean.info())

In [ ]:
df1_clean.dog_stage.value_counts()

### Issue 11

create extract days from `timestamp column`.

#### Define

* use the dt.strftime function to convert `timestamp` to `days`

In [ ]:
df1_clean.info()

In [ ]:
df1_clean['timestamp'] = pd.to_datetime(df1_clean['timestamp'])

#### Code

In [ ]:
# Extracting day from timestamp
df1_clean["day"] = df1_clean["timestamp"].dt.strftime("%a")

#dropping timestamp
df1_clean.drop('timestamp', axis=1, inplace=True)

#### Test

In [ ]:
df1_clean.info()

In [ ]:
df1_clean["day"].value_counts()

### Issue #12

* The three DataFrame should be merged as one.

#### Define
* Use the merge function to merge df1, df2, df3.

#### Code

In [ ]:
df = pd.merge(df1_clean, df2_clean, on='tweet_id', how='inner').merge(df3_clean)

#### Test

In [ ]:
df.info()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
# save the df to csv file
df.to_csv("..data/twitter_archive_master.csv", index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights

1. Relationship between dog `ratings`, `retweets`, `favorites`, `img_number`.
2. Explore the data description
3. Which day has the most Tweet
4. Relationship between Tweet day, and number of retweet and favorites.

In [ ]:
# make a copy of df dataset
df_clean = df.copy()

### Insights #1:

* explore the Relationship between dog `ratings`, `retweets`, `favorites`, `img_number`.

In [ ]:
# using a visual plot for this
sns.pairplot(df_clean, vars=["rating", "retweet_count",
                            "favorite_count", "img_num",
                            ], hue="day");

In [ ]:
# correlation table round off the 3 dp
df_clean.corr().round(3)

Looking at the visual plot and the correlation table above, considering the relationship between `retweeted_count` and `favorite_count`. It can be observed that there's a positive correlation between these two paramters. the correlation table shows a strong `0.913`. this means that the more retweets a tweet gets, the more favorites it will get.

### Insight #2:

* Explore the data description

In [ ]:
df_clean.describe()

* `img_num`: here the min img_num for a post that had an image is 1 and the maximium is 4.
*`retweet_count` and `favorite_count`: based on the mean, it can be observed that most tweets get a favorite than a retweet.

### Insight #3

* day with the most tweets

In [ ]:
df_clean_d = df_clean.day.value_counts()
df_clean_d

### Visualization

In [ ]:
df_clean_d.plot(kind='pie', figsize=(8,8));

Based on the `day column`, monday ranks as the day with the most tweets

### Insight #4

* Relationship between tweet day, retweet and favorite

In [ ]:
df_clean_drf=df_clean.groupby(["day"], as_index=False)["retweet_count", "favorite_count"].sum()
df_clean_drf.sort_values(by=["retweet_count"], ascending = False).head(7)

From our previous analysis, Monday ranked top with number of tweets. Apparently, though most tweets are made on Mondays, most retweets and favorites are made on Wednesday. so Wednesdays seem to be the most engaging day of WeRateDogs twitter page.
